In [1]:
from pymol import cmd
import numpy as np
import pandas as pd
import sqlite3
import os
import shutil
from plip.structure.preparation import PDBComplex

In [2]:
root = os.getcwd()
conn = sqlite3.connect('nanobio_data.db')
c = conn.cursor()

In [4]:
df = pd.read_sql('SELECT * FROM SimVina', conn, index_col='simvina_id')

In [6]:
query = df[(df.ligand_name == 'OLAPARIB_OPT') & (df.energy <= -10)]
#query = df.sample(4)
query

,ligand_name,file_name,count,x,y,z,energy,energypoints,protein_name,ligand_id,protein_id
simvina_id,,,,,,,,,,,
16974,OLAPARIB_OPT,"data\Results\OLAPARIB_OPT,-65,55,35.pdbqt",1,-65.499135,52.901242,28.740710,-10.4,-0.325000,6w01_prot,45,1
16975,OLAPARIB_OPT,"data\Results\OLAPARIB_OPT,-65,55,35.pdbqt",2,-65.297014,52.456291,28.234397,-10.3,-0.321875,6w01_prot,45,1
16976,OLAPARIB_OPT,"data\Results\OLAPARIB_OPT,-65,55,35.pdbqt",3,-62.663357,52.301510,27.971125,-10.2,-0.318750,6w01_prot,45,1


In [7]:
class state:
    def __init__(self, file_name=None, state=1):
        self.state = str(state)
        self.file_name = file_name
    #Create the .pdbqt with current state 
    def run(self):
        pdbqt = 'MODEL '
        with open(os.path.join(root, self.file_name)) as w:
            data = w.read().split('MODEL ')
            for x in data:
                if len(x) != 0 and x[0] in self.state:
                    pdbqt += x
        try: os.mkdir(os.path.join(os.getcwd(), 'cache'))
        except: pass
        open(os.path.join(os.getcwd(), 'cache', self.state + os.path.split(self.file_name)[-1]), 'w+').write(pdbqt)
    #Join protein-ligand in an unique pdb file  
    def join(self, protein):
        cmd.load(os.path.join(os.getcwd(), 'cache', self.state + os.path.split(self.file_name)[-1]))
        cmd.load(protein, os.path.split(protein)[-1])
        cmd.save(os.path.join(os.getcwd(), 'cache', os.path.split(protein)[-1].split('.')[0]+'_'+self.state+os.path.split(self.file_name)[-1].split('.')[0]+'.pdb'))
        cmd.reinitialize()
    #Delete the cache folder
    def clear(self):
        shutil.rmtree(os.path.join(os.getcwd(), 'cache'))
    #Delete all .pdbqt files in cache folder
    def clear_pdbqt(self):
        for delete in os.listdir(os.path.join(os.getcwd(), 'cache')):
            if delete != None and delete[-5:] == 'pdbqt':
                os.remove(os.path.join(os.getcwd(), 'cache', delete))

In [8]:
def dataReadid_protein(v): #SQL commands
    for row in c.execute(f'SELECT protein_loc FROM Protein WHERE protein_id = {v}'):
        return row

In [10]:
for ind in query.index:
    protein = os.path.join(os.getcwd(), dataReadid_protein(query['protein_id'][ind])[0]+'.pdbqt')
    l = state(query['file_name'][ind], query['count'][ind])
    l.run()
    l.join(protein)
l.clear_pdbqt()

 PyMOL not running, entering library mode (experimental)


In [36]:
for interation in os.listdir(os.path.join(os.getcwd(), 'cache')):
    my_mol = PDBComplex()
    my_mol.load_pdb(os.path.join(os.getcwd(), 'cache', interation)) # Load the PDB file into PLIP class
    my_bsid = str(my_mol).split('\n')[1]
    my_mol.analyze()
    my_interactions = my_mol.interaction_sets[my_bsid]
    vec = np.array([])
    
    saltbridge = [x.restype + str(x.resnr) + ':' + x.reschain for x in my_interactions.saltbridge_lneg] + [x.restype + str(x.resnr) + ':' + x.reschain for x in my_interactions.saltbridge_pneg]
    hbonds = [x.restype + str(x.resnr) + ':' + x.reschain for x in my_interactions.hbonds_ldon] +[x.restype + str(x.resnr) + ':' + x.reschain for x in my_interactions.hbonds_pdon]
    pistacking = [x.restype + str(x.resnr) + ':' + x.reschain for x in my_interactions.pistacking]
    pication = [x.restype + str(x.resnr) + ':' + x.reschain for x in my_interactions.pication_paro] + [x.restype + str(x.resnr) + ':' + x.reschain for x in my_interactions.pication_laro]
    hydrophobic_contacts = [x.restype + str(x.resnr) + ':' + x.reschain for x in my_interactions.hydrophobic_contacts]
    halogen_bonds = [x.restype + str(x.resnr) + ':' + x.reschain for x in my_interactions.halogen_bonds]
    water_bridges = [x.restype + str(x.resnr) + ':' + x.reschain for x in my_interactions.water_bridges]
    metal_complexes = [x.restype + str(x.resnr) + ':' + x.reschain for x in my_interactions.metal_complexes ]
    np.append(vec, saltbridge, axis=0)
    np.append(vec, hbonds, axis=0)
    np.append(vec, pistacking, axis=0)
    np.append(vec, pication, axis=0)
    np.append(vec, hydrophobic_contacts, axis=0)
    np.append(vec, halogen_bonds, axis=0)
    print(interation)
    print('saltbridge', saltbridge)
    
    print('hbonds', hbonds)
    
    print('pistacking', pistacking)
    
    print('pication', pication)
    
    print('hydrophobic_contacts', hydrophobic_contacts)
    
    print('halogen_bonds', halogen_bonds)
    
    print('water_bridges', water_bridges)
    
    print('metal_complexes', metal_complexes)
    print('\n\n')

6w01_prot_1OLAPARIB_OPT,-65,55,35.pdb
saltbridge []
hbonds ['LEU201:A', 'GLN202:A', 'LYS71:A', 'SER198:A']
pistacking []
pication []
hydrophobic_contacts ['ASN200:A', 'LEU252:A', 'LEU266:A', 'LYS277:A', 'VAL295:A']
halogen_bonds []
water_bridges []
metal_complexes []



6w01_prot_2OLAPARIB_OPT,-65,55,35.pdb
saltbridge []
hbonds ['LYS71:A', 'SER198:A', 'GLN202:A']
pistacking []
pication []
hydrophobic_contacts ['LEU201:A', 'PHE259:A', 'LEU266:A', 'LEU266:A', 'LYS277:A', 'TYR279:A', 'LEU201:A', 'ASP297:A']
halogen_bonds []
water_bridges []
metal_complexes []



6w01_prot_3OLAPARIB_OPT,-65,55,35.pdb
saltbridge []
hbonds ['THR275:A', 'LYS71:A', 'ASP297:A']
pistacking []
pication ['LYS90:A']
hydrophobic_contacts ['LEU201:A', 'LEU201:A', 'LEU266:A', 'VAL295:A', 'ASP297:A', 'LEU252:A', 'LYS277:A']
halogen_bonds ['ASN200:A']
water_bridges []
metal_complexes []



